In [11]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


In [5]:
data = pd.read_csv('../scraping/all_data_batch1_30to60k')

In [87]:
data

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,image,url
0,Soul Axiom Rebooted,2020-02-27,0,0,4,2,['Wales Interactive'],0.0,"['Adventure', 'Indie', 'Puzzle']","['Windows PC', 'Mac', 'Nintendo Switch']",...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/soul-axiom-reb...
1,Panic Station VR,2020-02-27,1,0,0,0,[],0.0,['Indie'],['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/panic-station-vr/
2,LoveR Kiss,2020-02-27,0,0,0,2,['Kadokawa Games'],0.0,"['Adventure', 'Visual Novel']","['PlayStation 4', 'Nintendo Switch']",...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/lover-kiss/
3,Boxman's Struggle,2020-02-27,1,0,1,1,['OneBlock'],0.0,"['Adventure', 'Indie', 'Platform']",['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/boxmans-struggle/
4,Kemono Heroes,2020-02-27,7,0,6,3,"['NIS America', 'MadGearGames']",3.0,"['Adventure', 'Indie', 'Platform']","['Nintendo Switch', 'Google Stadia']",...,0,1,1,1,0,0,1,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/kemono-heroes/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29954,Distraint,2015-10-21,283,5,88,22,['Jesse Makkonen'],3.2,"['Adventure', 'Indie', 'Puzzle']",['Windows PC'],...,2,16,20,59,51,28,7,8,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/distraint/
29955,Murder,2015-10-21,18,0,4,5,"['Peter Moorhead', 'Curve Digital']",2.0,"['Adventure', 'Indie', 'Point-and-Click', 'Puz...","['Windows PC', 'Mac', 'Linux']",...,3,4,3,0,0,1,0,1,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/murder--1/
29956,Yo-Kai Watch Puni-puni,2015-10-21,6,1,0,0,[],0.0,"['Puzzle', 'RPG']","['Android', 'iOS']",...,0,0,1,0,0,0,1,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/yo-kai-watch-p...
29957,Playing History - The Plague,2015-10-21,1,0,1,0,['Serious Games Interactive'],0.0,"['Adventure', 'Indie']","['Windows PC', 'Mac']",...,0,0,0,0,0,0,0,0,NaN,https://www.backloggd.com/games/playing-histor...


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29959 entries, 0 to 29958
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               29959 non-null  object 
 1   release_date        29959 non-null  object 
 2   plays               29959 non-null  object 
 3   playing             29959 non-null  object 
 4   backlogs            29959 non-null  object 
 5   wishlist            29959 non-null  object 
 6   developers          29959 non-null  object 
 7   avg_review          29959 non-null  float64
 8   genres              29959 non-null  object 
 9   platforms           29959 non-null  object 
 10  description         29134 non-null  object 
 11  total_reviews       29959 non-null  object 
 12  total_lists         29959 non-null  object 
 13  category            29959 non-null  object 
 14  main                29959 non-null  object 
 15  ratings_zero_five   29959 non-null  int64  
 16  rati

In [88]:
genre_df = data[['title', 'genres']]
developer_df = data[['title', 'developers']]
platform_df = data[['title', 'platforms']]

In [73]:
genre_df

,title,genres
0,Soul Axiom Rebooted,"['Adventure', 'Indie', 'Puzzle']"
1,Panic Station VR,['Indie']
2,LoveR Kiss,"['Adventure', 'Visual Novel']"
3,Boxman's Struggle,"['Adventure', 'Indie', 'Platform']"
4,Kemono Heroes,"['Adventure', 'Indie', 'Platform']"
...,...,...
29954,Distraint,"['Adventure', 'Indie', 'Puzzle']"
29955,Murder,"['Adventure', 'Indie', 'Point-and-Click', 'Puz..."
29956,Yo-Kai Watch Puni-puni,"['Puzzle', 'RPG']"
29957,Playing History - The Plague,"['Adventure', 'Indie']"


In [75]:
row = data['genres'][0]

In [142]:
def make_stringlist_list(string):
    '''This removes square brackets, and splits the string by comma to the create a list'''
    list_of_strings = string[2:-2].replace("'", '').split(',')
    return list_of_strings

def remove_whitespace(list_):
    '''This removes whitesapces from items within lists'''
    empty = []
    
    for i in range(len(list_)):
        item = list_[i].strip()
        empty.append(item)
    return empty

def clean_stringlists(df):
    '''For lists that were imported as strings, this removes square brackers and cleans up trailing whitespaces'''
    string_to_list_df = df.apply(make_stringlist_list)
    remove_whitespace_df = string_to_list_df.apply(remove_whitespace)
    
    return remove_whitespace_df

In [162]:
def make_list_columns_to_lists(df, columns):
    '''This returns a dataframe of columns where lists where imported as strings, and returns them to their list state'''
    cleaned_df = pd.DataFrame()
    
    for col in columns:
        cleaned = clean_stringlists(df[col])
        cleaned_df[col] = cleaned
    return cleaned_df

In [164]:
updated_df = make_list_columns_to_lists(data, ['genres', 'platforms'])

In [171]:
type(updated_df['genres'][0])

list

In [161]:
clean_stringlists(data['platforms'])

['Windows PC', 'Mac', 'Nintendo Switch']

In [85]:
genre_as_list = genre_df['genres'].apply(make_stringlist_list)

In [89]:
platforms_as_list = platform_df['platforms'].apply(make_stringlist_list)

In [93]:
developers_as_list = developer_df['developers'].apply(make_stringlist_list)

In [130]:
platforms_as_list[0][1]

' Mac'

In [133]:
platforms_as_list[0][0].strip()

'Windows PC'

In [134]:
remove_whitespace(platforms_as_list[0])

['Windows PC', 'Mac', 'Nintendo Switch']

In [136]:
platforms_as_list.apply(remove_whitespace)

['Windows PC', 'Mac', 'Nintendo Switch']

In [28]:
mlb = MultiLabelBinarizer()


transfomed = mlb.fit_transform(genre_df['genres'])

In [29]:
pd.DataFrame(transfomed, columns=mlb.classes_)

,,&,',",",-,/,A,B,C,F,...,o,p,r,s,t,u,v,w,y,z
0,1,0,1,1,0,0,1,0,0,0,...,0,0,1,0,1,1,1,0,0,1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,1,0,0,1,0,0,0,...,1,0,1,1,1,1,1,0,0,0
3,1,0,1,1,0,0,1,0,0,0,...,1,0,1,0,1,1,1,0,0,0
4,1,0,1,1,0,0,1,0,0,0,...,1,0,1,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29954,1,0,1,1,0,0,1,0,0,0,...,0,0,1,0,1,1,1,0,0,1
29955,1,0,1,1,1,0,1,0,1,0,...,1,0,1,0,1,1,1,0,1,1
29956,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
29957,1,0,1,1,0,0,1,0,0,0,...,0,0,1,0,1,1,1,0,0,0


In [20]:
len(transfomed[0])

46